#### **Question 1**<br>
- Implement the Perceptron algorithm from scratch in Python.<br>
- Initialize the weights with [0 0 0] and a learning rate of 0.0001.<br>
- For each iteration, calculate the output of the Perceptron for each input in the training set.<br>
- Use MSE to computer the error for all samples<br>
- Update the weights using the gradient descent procedure.<br>
- Repeat the above steps until the Perceptron converges or a maximum number of iterations is reached.<br>
- Test the trained Perceptron on a separate test set, explain how you came up with the test set.<br>
- Use the step function as an  activation function in the output layer<br>

Use the IRIS Dataset for the above, considering all four features: sepal length, sepal width, petal length, and petal width, but only two classes -  Setosa, and Versicolor.  Drop the feature vectors of the other class. <br>



In [1]:
import numpy as np
import pandas as pd

In [2]:
dataset = pd.read_csv('Iris Dataset - iris.csv')
dataset.head(3)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Convert species into numerical labels
label_encoder = LabelEncoder()
dataset['Species'] = label_encoder.fit_transform(dataset['Species'])

# Shuffle the dataset and split it into training, validation, and test sets
train_val, test = train_test_split(dataset, test_size=0.1, random_state=42, shuffle=True)
train, val = train_test_split(train_val, test_size=0.1, random_state=42, shuffle=True)

# Display the sizes of the splits
print(f"Training set size: {train.shape[0]}")
print(f"Validation set size: {val.shape[0]}")
print(f"Test set size: {test.shape[0]}")

Training set size: 121
Validation set size: 14
Test set size: 15


In [4]:
train.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
1,2,4.9,3.0,1.4,0.2,0
133,134,6.3,2.8,5.1,1.5,2
95,96,5.7,3.0,4.2,1.2,1
132,133,6.4,2.8,5.6,2.2,2
125,126,7.2,3.2,6.0,1.8,2


#### Perceptron single layer single output without non linearity

In [56]:
class Perceptron:
    def __init__(self):
        self.w = np.zeros(4)
        self.b = 0
        self.learning_Rate = 0.001
    
    def forward_propogation(self,x):
        y_hat = np.dot(self.w,x)+self.b 
        return y_hat
        
    def calculate_gradient(self,y_hat,y,x):
        dw = (y_hat - y) * x
        db = (y_hat - y)
        return dw,db
        
    def apply_gradient_descent(self,dw,db):
        self.w -= self.learning_Rate * dw
        self.b -= self.learning_Rate * db
        
    def one_full_pass(self,train,val):
        x_train = np.array(train[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']])
        y_train = np.array(train['Species'])
        dataset_size = x_train.shape[0]
        
        for i in range(dataset_size):
            y_hat = self.forward_propogation(x_train[i])
            dw,db = self.calculate_gradient(y_hat,y_train[i],x_train[i])
            self.apply_gradient_descent(dw,db)
        
        x_val = np.array(val[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']])
        y_val = np.array(val['Species'])
        val_size = x_val.shape[0]
        error = 0
        
        for i in range(val_size):
            error += (y_val[i] - self.forward_propogation(x_val[i])) ** 2
            
        error /= val_size
        
        return error
        
    def train(self,epochs,train,val):
        for i in range(epochs):
            val_error = self.one_full_pass(train,val)
            print(f"Completed epoch - {i} , val error:{val_error:.4f}")
        
        
    def evaluate(self,test):
        x_test = np.array(test[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']])
        y_test = np.array(test['Species'])
        test_size = x_test.shape[0]
        count = 0
        for i in range(test_size):
            y_hat = round(self.forward_propogation(x_test[i]))
            if y_hat == y_test[i]:
                count+=1
            
        accuracy = count / test_size
        print(f"Accuracy on TEST DATA ONLY : {accuracy}")
            
   
P  = Perceptron()
P.train(10,train,val)
P.evaluate(test)

Completed epoch - 0 , val error:0.1980
Completed epoch - 1 , val error:0.1432
Completed epoch - 2 , val error:0.1110
Completed epoch - 3 , val error:0.0921
Completed epoch - 4 , val error:0.0810
Completed epoch - 5 , val error:0.0745
Completed epoch - 6 , val error:0.0705
Completed epoch - 7 , val error:0.0681
Completed epoch - 8 , val error:0.0667
Completed epoch - 9 , val error:0.0657
Accuracy on TEST DATA ONLY : 1.0


In [57]:
print(f"Model weights after training : {P.w}")
print(f"Model bias : {P.b}")

Model weights after training : [-0.00049592 -0.13065826  0.34317196  0.17606454]
Model bias : -0.024905290070315243


In [58]:
x_train = np.array(train[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']])
y_train = np.array(train['Species'])

In [62]:
# Sample test
i = 36
x = x_train[i]
y = y_train[i]
print(f"X = {x},Y = {y}")
y_hat = round(P.forward_propogation(x))
print(f"Predicted = {y_hat}")

X = [5.  3.3 1.4 0.2],Y = 0
Predicted = 0


In [60]:
x_train

array([[4.9, 3. , 1.4, 0.2],
       [6.3, 2.8, 5.1, 1.5],
       [5.7, 3. , 4.2, 1.2],
       [6.4, 2.8, 5.6, 2.2],
       [7.2, 3.2, 6. , 1.8],
       [4.4, 2.9, 1.4, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [6. , 3.4, 4.5, 1.6],
       [5.7, 4.4, 1.5, 0.4],
       [4.8, 3.4, 1.6, 0.2],
       [6.7, 3. , 5.2, 2.3],
       [7.3, 2.9, 6.3, 1.8],
       [6.3, 2.7, 4.9, 1.8],
       [6.7, 2.5, 5.8, 1.8],
       [5.7, 2.8, 4.1, 1.3],
       [5.5, 2.5, 4. , 1.3],
       [5.2, 2.7, 3.9, 1.4],
       [6. , 2.2, 5. , 1.5],
       [7.6, 3. , 6.6, 2.1],
       [6.3, 3.3, 4.7, 1.6],
       [5.5, 2.4, 3.8, 1.1],
       [5.5, 4.2, 1.4, 0.2],
       [5.2, 4.1, 1.5, 0.1],
       [5.1, 3.7, 1.5, 0.4],
       [6.7, 3.1, 5.6, 2.4],
       [5.7, 2.5, 5. , 2. ],
       [4.6, 3.6, 1. , 0.2],
       [6.4, 2.9, 4.3, 1.3],
       [7.7, 2.8, 6.7, 2. ],
       [5.2, 3.5, 1.5, 0.2],
       [6.4, 2.7, 5.3, 1.9],
       [6.7, 3. , 5. , 1.7],
       [4.7, 3.2, 1.3, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5.5, 2